In [4]:
import networkx as nx
import numpy as np
from src.diffusion_models import independent_cascade_model, linear_threshold_model
from src.polarization import optimize_tau 
from src.graph import random_color_graph, create_polarized_graph
from src.seed import seed_random, seed_degree

In [7]:
def get_red_and_blue_nodes(G):
    red_nodes = [node for node in G.nodes if G.nodes[node]["color"] == "red"]
    blue_nodes = [node for node in G.nodes if G.nodes[node]["color"] == "blue"]
    return red_nodes, blue_nodes

def add_edges_using_optimize_tau(G, C, k, perc_k):
    """Add edges to the graph G using the optimize_tau function."""
    red_nodes, blue_nodes = get_red_and_blue_nodes(G)
    F_set, _ = optimize_tau(C, G, k, red_nodes, blue_nodes, perc_k)
    G.add_edges_from(F_set)

def run_diffusion_with_same_seed(
    create_graph_func,
    num_nodes,
    intra_group_connectness,
    inter_group_connectness,
    add_edges_func,
    diffusion_model_func,
    seed_func,
    seed_count,
    threshold=None,
    k=5,
    perc_k=5,
):
    # Create original graph
    original_graph = create_graph_func(
        num_nodes, intra_group_connectness, inter_group_connectness
    )
    # color the graph
    random_color_graph(original_graph)

    # Select seeds
    seeds = seed_func(original_graph, seed_count)

    # Generate thresholds for each node if using linear threshold model
    if diffusion_model_func == linear_threshold_model:
        thresholds = {node: np.random.uniform(0, 1) for node in original_graph.nodes}
    else:
        thresholds = threshold

    # Add new edges to the graph
    modified_graph = original_graph.copy()
    add_edges_func(modified_graph, C=seeds, k=k, perc_k=perc_k)

    print("Original graph")
    print("Number of nodes: ", original_graph.number_of_nodes())
    print("Number of edges: ", original_graph.number_of_edges())

    print("Modified graph")
    print("Number of nodes: ", modified_graph.number_of_nodes())
    print("Number of edges: ", modified_graph.number_of_edges())

    # Run diffusion on original graph
    result_original = diffusion_model_func(original_graph, seeds, thresholds)

    # Run diffusion on modified graph
    result_modified = diffusion_model_func(modified_graph, seeds, thresholds)

    return result_original, result_modified

In [8]:
# Example usage
result_original, result_modified = run_diffusion_with_same_seed(
    create_polarized_graph,
    num_nodes=500,
    intra_group_connectness=0.2,
    inter_group_connectness=0.01,
    add_edges_func=add_edges_using_optimize_tau,
    diffusion_model_func=linear_threshold_model,
    seed_func=seed_random,
    seed_count=80,
    threshold=0.5,
    k=400,
    perc_k=10,
)
result_original[0], result_modified[0]

100%|██████████| 50/50 [00:02<00:00, 23.11it/s]

{(15, 39), (0, 109), (178, 109), (0, 118), (8, 0), (15, 109), (109, 95), (109, 40), (15, 118), (8, 15), (0, 197), (178, 197), (109, 0), (15, 197), (109, 15), (0, 181), (79, 114), (79, 178), (178, 181), (0, 199), (15, 181), (178, 199), (15, 199), (79, 180), (0, 134), (79, 15), (0, 79), (178, 134), (109, 114), (178, 79), (109, 178), (79, 140), (0, 39), (178, 39), (79, 167), (178, 118), (0, 8), (178, 8), (0, 26), (15, 8), (109, 180), (178, 26), (15, 26), (109, 140), (79, 95), (79, 40), (79, 0), (15, 134), (8, 95), (8, 40)}
4106
Original graph
Number of nodes:  200
Number of edges:  4058
Modified graph
Number of nodes:  200
Number of edges:  4106


(200, 200)